In [1]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
#import statements

In [2]:
#data is imported
df = pandas.read_csv("train.csv")
#irrelavent data is removed
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
#NaNs are removed
df = df.dropna()
#testing and training sets are split
train, test = train_test_split(df, test_size=0.2, random_state=4)

entropy $H(Y)$, conditional-entropy $H(Y|X)$, information gain $IG(X)$

$H(Y) = - \sum^K_{i=1} P(Y=y_i)\lg P(Y=y_i)$

$H(Y|X) = - \sum^V_{j=1} \biggl( P(X=x_j) \sum^K_{i=1} P(Y=y_i|X=x_j)\lg P(Y=y_i|X=x_j) \biggr)$

$IG(X) = H(Y)-H(Y|X)$

In [3]:
#this function takes a value and returns the product of that and it's base 2 log
#I use this function to make the following functions more readable
def helper(a):
    if (a == 1) or (a == 0):
        return 0
    return (a * np.log2(a))
#this function calculates the entropy(H(Y)) of the dataframe dataset
#classname will be 'Survive'
#the PosClassPrior/NegClassPrior will represent the prior probability of the positive/negative class
def entropy(dataset, classname):
    PosClassPrior = len(dataset[dataset[classname] == 1]) / len(dataset)
    NegClassPrior = 1 - PosClassPrior
    return 0 - (helper(PosClassPrior) + helper(NegClassPrior))
#this function calculates the conditional-entropy(H(Y|X)) of the dataframe dataset
#predictors are a list of strings. each string contains the conditional that the feature = some value
def conditionalEntropy(dataset, classname, predictors):
    #total is the variable we store the conditional entropy during the summations
    total = 0.0
    #a temporary dataframe holding a subset of our data
    tempDF = None
    #these 3 variables represent the predictor prior probability P(X=x), and 
    #the positive and negative class conditional densities P(Y=y|X=x)
    predPrior = 0.0
    PosClassCond = 0.0
    NegClassCond = 0.0
    #I iterate through every feature in the dataset
    for predictor in predictors:
        #here the subset of the data where feature = value is stored in tempDF
        #this if statement avoids a division by zero error
        tempDF = dataset.query(predictor)
        if(len(tempDF) > 0):
            #here the values are set using the afformentioned variables according to the formula
            predPrior = len(tempDF) / len(dataset)
            PosClassCond = len(tempDF[tempDF['Survived'] == 1]) / len(tempDF)
            NegClassCond = len(tempDF[tempDF['Survived'] != 1]) / len(tempDF)
            #and it is added to the total before iterating to the next value in 
            #the outer summation of the conditional entropy formula
            total -= predPrior * (helper(NegClassCond) + helper(PosClassCond))
    return total
#here the info gain is calculated and rounded to the tenth decimal place
#I round because any difference less than that is negligable and I want to save space
def infoGain(dataset, classname, predictors):
    a = entropy(dataset, classname) - conditionalEntropy(dataset, classname, predictors)
    return round(a,10)

Gini Index $I_G(p)$. Best-Case 1 or 0, worst-case $\frac 1 2$.

$p_+ = \frac{\text{items in the set}\in C_+}{\text{items in the set}}$

$I_G(p) = 1 - \sum^K_{j=1} p_j^2$

In [4]:
#calculates the gini index
def giniIndex(dataset):
    #this if statement avoids error but should never occur
    if len(dataset) == 0:
        return 0.5
    #positive and negatice represent the class prior probabilties
    positive = len(dataset[dataset['Survived'] > 0]) / len(dataset)
    negative = 1.0 - positive
    #k=2 so this is the whole gini index summation and formula
    return 1.0 - ((negative)**2 + (positive)**2)

In [5]:
#this function gathers all the query strings for each feature in the data and their set of values
def getAllQs(data):
    #Querys is a list of lists of strings
    ##each list contains the conditionals for all unique values(strings) of that list's feature
    #prev contains the previous value for certain features so they can be treated as a range if the testing data
    #contains a possible value for such features not in the training data
    Querys = []
    prev = 0.0
    #this iterates through every column/feature in the data/dataframe except for the target column
    for col in data.columns[1:]:
        Querys.append([])
        prev = -0.1
        #removes duplicates of each value for the feature in the samples
        l=list(set(data[col]))
        #sorts those values
        l.sort()
        #iterates through each unique value: v
        for v in l:
            #appends the feature to query the if statements are for syntax depending on the datatype of v
            #they also update prev to be the current value after the proper range is copied for age/fare
            if(col == 'Sex' or col == 'Embarked'):
                Querys[-1].append('%s == \'%s\''%(col,v))
            elif(col == 'Age' or col == 'Fare'):
                Querys[-1].append('%s <= %f and %s > %f'%(col,v,col,prev))
                prev = v
            else:
                Querys[-1].append('%s == %s'%(col,v))
    return Querys

In [6]:
#this functino returns a boolean indicating if all features match
def all_features_match(datas):
    #goes through each feature if the set of unique values has len > 1 there are non-matching features
    for col in datas.columns[1:]:
        if len(set(datas[col])) > 1:
            return False
    return True

In [7]:
#this function builds our decision tree classifier as a list of lists
#each list except for the "root" contains a conditional for the value of a feature and either the child nodes
#or class prediction
#max depth is a counter for recursion and gini indicates using gini index(=1) or information gain(=0)
def BuildTree(datas, maxdepth, gini):
    #checks if all target values belong to one class or if the features all match and returns the majority class label
    if (0 == len(datas[datas['Survived'] == 1])):
        return 0.0#a leafnode that predicts this
    elif (len(datas) == len(datas[datas['Survived'] == 1])):
        return 1.0#a leafnode that predicts this
    if (all_features_match(datas) or maxdepth == 0):
        return len(datas[datas['Survived'] == 1])/len(datas)#a leafnode that predicts this
    #these 3 variables store the list of querries, the nd-array containing each feature's list of querries+index
    #and a counter to keep track of the index within the for loop
    Qs = getAllQs(datas)
    store = np.zeros((len(Qs),2))
    index = -1
    #iterates through the values of Qs storing the gini-index or info-gain of each feature in store
    for q in Qs:
        index += 1
        store[index][0] = index
        if(gini == 1):
            store[index][1] = giniIndex(datas)
        else:
            store[index][1] = infoGain(datas, 'Survived', q)
            store[index][1] /= np.sqrt(len(q))#this is a penalty I added
            #I added this penalty because the there is a bias favoring features with many unique values
            #this becomes a problem because it is not necessarily the best split
    #iterates through every row in store
    for i,j in zip(store.T[0],store.T[1]):
        #i is the index j is corresponding the GI or IG value
        i = int(i)
        #here we have found the highest value
        if (j == max(set(store.T[1]))):
            ##print(Qs[i][0])
            child = []#the child nodes the len of this list = the number of unique values/ranges in the feature
            for featureValue in Qs[i]:#iterates through each value for the feature
                tempstr = featureValue.split(' ')[0]#this tempstr is the feature's name
                tempdataf = datas.query(featureValue).drop(columns=tempstr)#this temp data is a subset of out data
                #all values in tempdef have feature: tempstr equal to feature value
                #the feature is then dropped because it's irrelavent now
                child.append([featureValue, BuildTree(tempdataf, maxdepth-1, gini)])
                #the function is recursively called with the subset as the data and max depth decremented
                #finally the child is added to the tree this will happen for every unique feature value
            return child
    print("code should never reach here error!")

In [8]:
#this function returns the accuracy of my decision tree using a test dataframe
def testTree(acc, test, tree):
    tempData = None
    #if tree depth=0
    if(type(tree)==type(0.1)):
        return len(test[test['Survived'] == tree])/len(test)
    #otherwise it iterates through tree using the conditionals to seperate the test dataset by feature value
    for leaf in tree:
        #gets subset storing it in tempdata like in earlier functions
        tempData = test.query(leaf[0])
        if(len(tempData) == 0):
            continue
        elif(type(leaf[1]) == type(0.5)):#this means the leaf is a class prediction
            ##print('%s %f'%(list(tempData['Survived']),leaf[1]))
            if(leaf[1]>0.5):#accuracy is calculated and added
                acc += len(tempData[tempData['Survived'] == 1])
            else:
                acc += len(tempData[tempData['Survived'] != 1])
        else:#there are more branches that I use recursion to traverse
            acc += testTree(0, tempData, leaf[1])
    return acc#accuracy is returned

In [9]:
#here I print the length of the testing set call my functions and print my accuracy
print(len(test))
#This tree uses information gain with a max depth of 2
mytree = BuildTree(train, 2, False)
print(testTree(0, test, mytree)/len(test))
#This tree uses gini index with a max depth of 2
mytree2 = BuildTree(train, 2, True)
print(testTree(0, test, mytree2)/len(test))

#here I write my tree to a file not necessary for this assignment but helpful for visualization and debugging
def printTree(tree, ver, branch, fi):
    tempData = None
    ind = ''
    if(type(tree)==type(0.1)):
        myfile.write("%f\n"%tree)
        return tree
    for leaf in tree:
        ind = ''
        tempData = ver.query(leaf[0])
        if(type(leaf[1]) == type(0.5)):
            for b in range(branch):
                ind += '__'
            myfile.write("%s%s\n%s%s%s\n"%(ind, leaf[0], ind, ind, leaf[1]))
        else:
            for b in range(branch):
                myfile.write('__')
            myfile.write("%s,\n"%leaf[0])
            printTree(leaf[1], tempData, branch+1, fi)
myfile = open("./3tree.txt", "w")
try:
    printTree(mytree2, train, 0, myfile)
finally:
    myfile.close()

143
0.8181818181818182
0.8111888111888111
